In [1]:
# Import spacy, import the large english model (this is 600 MB), and disable tagger, ner and lemmatizer.
# Download the large English model ~ 600 MB if you have not done so: `python -m spacy download en_core_web_lg`.
import spacy
nlp = spacy.load('en_core_web_lg',disable=["tagger", "ner", "lemmatizer"])

In [2]:
# Read in the text file with the read_file function.
def read_file(filepath):
    """
    Reads in a text file from the directory and saves the file contents to a variable.
    
    Args:
        text (str): The input text to read and save
        
    Returns: 
        A string containing the file contents.
    """
    with open(filepath) as f:
        str_text = f.read()
    return str_text

## Tokenize and Clean Text

In [3]:
# Use the separate_punc function to remove the puncutation. 
def separate_punc(md_text):
    """
    Retrieves all the words in the text without punctuation 

    Args:
        text (str): The input text from which words are extracted witout punctuation.

    Returns:
        list: A list of words

    """
    # Create a list comprehension to get only the tokens, i.e., words in the text.
    return [token.text.lower() for token in nlp(md_text) \
            if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [4]:
# Pass in the first four chapters of Moby Dick to the read_file function.
md_text = read_file('Resources/moby_dick_four_chapters.txt')
# Clean and tokenize the text using the separate_punc function.
tokens = separate_punc(md_text)

In [5]:
# Get the length of the tokens list.
len(tokens)

11338

In [6]:
# Look over the tokems to make sure all theo punctuation has been reomved.  
# Some punctuation wasn't removed, i.e., "?--".
print(tokens[:300])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i', 'thought', 'i', 'would', 'sail', 'about', 'a', 'little', 'and', 'see', 'the', 'watery', 'part', 'of', 'the', 'world', 'it', 'is', 'a', 'way', 'i', 'have', 'of', 'driving', 'off', 'the', 'spleen', 'and', 'regulating', 'the', 'circulation', 'whenever', 'i', 'find', 'myself', 'growing', 'grim', 'about', 'the', 'mouth', 'whenever', 'it', 'is', 'a', 'damp', 'drizzly', 'november', 'in', 'my', 'soul', 'whenever', 'i', 'find', 'myself', 'involuntarily', 'pausing', 'before', 'coffin', 'warehouses', 'and', 'bringing', 'up', 'the', 'rear', 'of', 'every', 'funeral', 'i', 'meet', 'and', 'especially', 'whenever', 'my', 'hypos', 'get', 'such', 'an', 'upper', 'hand', 'of', 'me', 'that', 'it', 'requires', 'a', 'strong', 'moral', 'principle', 'to', 'prevent', 'me', 'from', '

In [7]:
# How many tokens contain "?--"?
count = 0
for token in tokens:
    if "?--" in token:
        count +=1
print(count)

6


The next step is to determine how long of token sequence we want to predict the next token. 
This also affects our model's accuracy. 
Hiakus are three token sequences and lyrics are seven token sequences. 
We want to create 25 token sequences and then predict the next token, number 26.

## Create Sequences of Tokens

In [9]:
# Organize into sequences of tokens.
# Use 25 words to predict the 26th word using "+1".
train_len = 26 # = 25 training words plus one target word.

# Create an empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    # Range is 26 to 11,338
    # Get the train_len amount of characters 
    # First iteration gets the words from 0:26
    # The second iteration getst the words from 1:27, etc.
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [10]:
# The first 26 words [0:26]
print(text_sequences[0])
# The next 26 words [1:27]
print(text_sequences[1]) 
# The next 26 words [2:28]. 
print(text_sequences[2])

['call', 'me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on']
['me', 'ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore']
['ishmael', 'some', 'years', 'ago', 'never', 'mind', 'how', 'long', 'precisely', 'having', 'little', 'or', 'no', 'money', 'in', 'my', 'purse', 'and', 'nothing', 'particular', 'to', 'interest', 'me', 'on', 'shore', 'i']


In [11]:
# Join the first 26 words. 
print(' '.join(text_sequences[0]))
# Join the next 26 words. 
print(' '.join(text_sequences[1]))
# And, join the next 26 words. 
print(' '.join(text_sequences[2]))

call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on
me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore
ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i


In [12]:
# The list of text_sequences should be 26 less than the total tokens.
len(text_sequences)

11312

## Perform Tokenization with Keras

In [13]:
# Import the Keras tokenization to format the data from words into a numerical format.
from keras.preprocessing.text import Tokenizer

2023-11-13 10:21:04.266172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# Initialize the Keras Tokenizer class 
tokenizer = Tokenizer()
# Map each word with an index.
tokenizer.fit_on_texts(text_sequences)

In [15]:
# Get the dictionary mapping of words to their indices
print(tokenizer.word_index)

{'the': 1, 'a': 2, 'and': 3, 'of': 4, 'i': 5, 'to': 6, 'in': 7, 'it': 8, 'that': 9, 'he': 10, 'his': 11, 'was': 12, 'but': 13, 'me': 14, 'with': 15, 'as': 16, 'at': 17, 'this': 18, 'you': 19, 'is': 20, 'all': 21, 'for': 22, 'my': 23, 'on': 24, 'be': 25, "'s": 26, 'not': 27, 'from': 28, 'there': 29, 'one': 30, 'up': 31, 'what': 32, 'him': 33, 'so': 34, 'bed': 35, 'now': 36, 'about': 37, 'no': 38, 'into': 39, 'by': 40, 'were': 41, 'out': 42, 'or': 43, 'harpooneer': 44, 'had': 45, 'then': 46, 'have': 47, 'an': 48, 'upon': 49, 'little': 50, 'some': 51, 'old': 52, 'like': 53, 'if': 54, 'they': 55, 'would': 56, 'do': 57, 'over': 58, 'landlord': 59, 'thought': 60, 'room': 61, 'when': 62, 'could': 63, "n't": 64, 'night': 65, 'here': 66, 'head': 67, 'such': 68, 'which': 69, 'man': 70, 'did': 71, 'sea': 72, 'time': 73, 'other': 74, 'very': 75, 'go': 76, 'these': 77, 'more': 78, 'though': 79, 'first': 80, 'sort': 81, 'said': 82, 'last': 83, 'down': 84, 'most': 85, 'been': 86, 'never': 87, 'your':

In [16]:
# Get the dictionary of words and the number of times they appear in the text.
print(tokenizer.word_counts) 

OrderedDict([('call', 27), ('me', 2471), ('ishmael', 133), ('some', 758), ('years', 135), ('ago', 84), ('never', 449), ('mind', 164), ('how', 321), ('long', 374), ('precisely', 37), ('having', 142), ('little', 767), ('or', 950), ('no', 1003), ('money', 120), ('in', 5647), ('my', 1786), ('purse', 71), ('and', 9646), ('nothing', 281), ('particular', 152), ('to', 6497), ('interest', 24), ('on', 1716), ('shore', 26), ('i', 7150), ('thought', 676), ('would', 702), ('sail', 104), ('about', 1014), ('a', 10377), ('see', 416), ('the', 15540), ('watery', 26), ('part', 234), ('of', 8287), ('world', 234), ('it', 4238), ('is', 1950), ('way', 390), ('have', 806), ('driving', 26), ('off', 416), ('spleen', 26), ('regulating', 26), ('circulation', 26), ('whenever', 130), ('find', 78), ('myself', 416), ('growing', 26), ('grim', 26), ('mouth', 130), ('damp', 78), ('drizzly', 26), ('november', 26), ('soul', 78), ('involuntarily', 52), ('pausing', 52), ('before', 364), ('coffin', 104), ('warehouses', 52), 

In [17]:
# What is the size of the vocabulary
vocabulary_size = len(tokenizer.word_counts)
print(vocabulary_size)

2718


In [18]:
# Encode each word in the text_sequences to the indices. 
sequences = tokenizer.texts_to_sequences(text_sequences)

In [65]:
# Get the encoded indices for the the first 26 words
print(sequences[0])

[956, 14, 263, 51, 261, 408, 87, 219, 129, 111, 954, 260, 50, 43, 38, 314, 7, 23, 546, 3, 150, 259, 6, 2713, 14, 24]


In [20]:
# Get the word associated with the indices for the first sequence.
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [21]:
# Get the word associated with a specific index.
tokenizer.index_word.get(956)

'call'

In [22]:
# Get the number of times the word "call" appears in the text.
count = 0
index_to_word = {index: word for word, index in tokenizer.word_index.items()}
for sequence in sequences:
    words = [index_to_word.get(index, '') for index in sequence]
    count += words.count(tokenizer.index_word.get(956, ''))

print(count)

27


## Convert the List of Sequences to Arrays.

In [23]:
# Import numpy to convert the list of sequences to arrays.
import numpy as np

In [24]:
# Convert the all 26 word list of lists to arrays.
num_sequences = np.array(sequences)
print(num_sequences)

[[ 956   14  263 ... 2713   14   24]
 [  14  263   51 ...   14   24  957]
 [ 263   51  261 ...   24  957    5]
 ...
 [ 952   12  166 ...  262   53    2]
 [  12  166 2712 ...   53    2 2718]
 [ 166 2712    3 ...    2 2718   26]]


In [25]:
# Get the length of the array
len(num_sequences)

11312

In [26]:
# Get the first array.
print(num_sequences[0])

[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  314    7   23  546    3  150  259    6 2713   14   24]


## Create input sequences and one-hot encode the target variable.

In [66]:
# Import the to_categorical function to convert the arrays to binary values.
import keras
from keras.utils import to_categorical

In [67]:
# Get the first 25 numbers from each array. These will be our "X". 
for sequence in num_sequences[:,:-1]:
    print(sequence)

[ 956   14  263   51  261  408   87  219  129  111  954  260   50   43
   38  314    7   23  546    3  150  259    6 2713   14]
[  14  263   51  261  408   87  219  129  111  954  260   50   43   38
  314    7   23  546    3  150  259    6 2713   14   24]
[ 263   51  261  408   87  219  129  111  954  260   50   43   38  314
    7   23  546    3  150  259    6 2713   14   24  957]
[  51  261  408   87  219  129  111  954  260   50   43   38  314    7
   23  546    3  150  259    6 2713   14   24  957    5]
[ 261  408   87  219  129  111  954  260   50   43   38  314    7   23
  546    3  150  259    6 2713   14   24  957    5   60]
[ 408   87  219  129  111  954  260   50   43   38  314    7   23  546
    3  150  259    6 2713   14   24  957    5   60    5]
[  87  219  129  111  954  260   50   43   38  314    7   23  546    3
  150  259    6 2713   14   24  957    5   60    5   56]
[ 219  129  111  954  260   50   43   38  314    7   23  546    3  150
  259    6 2713   14   24  957   

[ 683   65 1425  212    3 1426    5  267   38   30    7    1  108   15
 1427 1428    5   45 1429   23  473    3  119 1430   31]
[  65 1425  212    3 1426    5  267   38   30    7    1  108   15 1427
 1428    5   45 1429   23  473    3  119 1430   31    2]
[1425  212    3 1426    5  267   38   30    7    1  108   15 1427 1428
    5   45 1429   23  473    3  119 1430   31    2  320]
[ 212    3 1426    5  267   38   30    7    1  108   15 1427 1428    5
   45 1429   23  473    3  119 1430   31    2  320 1431]
[   3 1426    5  267   38   30    7    1  108   15 1427 1428    5   45
 1429   23  473    3  119 1430   31    2  320 1431    4]
[1426    5  267   38   30    7    1  108   15 1427 1428    5   45 1429
   23  473    3  119 1430   31    2  320 1431    4 1432]
[   5  267   38   30    7    1  108   15 1427 1428    5   45 1429   23
  473    3  119 1430   31    2  320 1431    4 1432  684]
[ 267   38   30    7    1  108   15 1427 1428    5   45 1429   23  473
    3  119 1430   31    2  320 14

[ 56 786  49 100   1  44 149  25   3   9  54  10   1  59 339  45  38  74
 108  22  14   3   1  44  12]
[786  49 100   1  44 149  25   3   9  54  10   1  59 339  45  38  74 108
  22  14   3   1  44  12  27]
[  49  100    1   44  149   25    3    9   54   10    1   59  339   45
   38   74  108   22   14    3    1   44   12   27 1843]
[ 100    1   44  149   25    3    9   54   10    1   59  339   45   38
   74  108   22   14    3    1   44   12   27 1843 1844]
[   1   44  149   25    3    9   54   10    1   59  339   45   38   74
  108   22   14    3    1   44   12   27 1843 1844  140]
[  44  149   25    3    9   54   10    1   59  339   45   38   74  108
   22   14    3    1   44   12   27 1843 1844  140  184]
[ 149   25    3    9   54   10    1   59  339   45   38   74  108   22
   14    3    1   44   12   27 1843 1844  140  184  107]
[  25    3    9   54   10    1   59  339   45   38   74  108   22   14
    3    1   44   12   27 1843 1844  140  184  107 1845]
[   3    9   54   10    1 

[   6  277   42    3   91   14  100    3   32   18   44   20    3  203
    5 2128   25    7   21 2129 2130    6  513    1   65]
[ 277   42    3   91   14  100    3   32   18   44   20    3  203    5
 2128   25    7   21 2129 2130    6  513    1   65   15]
[  42    3   91   14  100    3   32   18   44   20    3  203    5 2128
   25    7   21 2129 2130    6  513    1   65   15   33]
[   3   91   14  100    3   32   18   44   20    3  203    5 2128   25
    7   21 2129 2130    6  513    1   65   15   33    3]
[  91   14  100    3   32   18   44   20    3  203    5 2128   25    7
   21 2129 2130    6  513    1   65   15   33    3    7]
[  14  100    3   32   18   44   20    3  203    5 2128   25    7   21
 2129 2130    6  513    1   65   15   33    3    7    1]
[ 100    3   32   18   44   20    3  203    5 2128   25    7   21 2129
 2130    6  513    1   65   15   33    3    7    1   80]
[   3   32   18   44   20    3  203    5 2128   25    7   21 2129 2130
    6  513    1   65   15   33   

[  43   74 2394   69   11  177 2395   37    7    1   85  754  385   17
   83 2396    1  196   10  174    1  404   31   75 2397]
[  74 2394   69   11  177 2395   37    7    1   85  754  385   17   83
 2396    1  196   10  174    1  404   31   75 2397    3]
[2394   69   11  177 2395   37    7    1   85  754  385   17   83 2396
    1  196   10  174    1  404   31   75 2397    3 2398]
[  69   11  177 2395   37    7    1   85  754  385   17   83 2396    1
  196   10  174    1  404   31   75 2397    3 2398    8]
[  11  177 2395   37    7    1   85  754  385   17   83 2396    1  196
   10  174    1  404   31   75 2397    3 2398    8  144]
[ 177 2395   37    7    1   85  754  385   17   83 2396    1  196   10
  174    1  404   31   75 2397    3 2398    8  144    7]
[2395   37    7    1   85  754  385   17   83 2396    1  196   10  174
    1  404   31   75 2397    3 2398    8  144    7   11]
[  37    7    1   85  754  385   17   83 2396    1  196   10  174    1
  404   31   75 2397    3 2398   

[  78    1 2676  930    9  109  128 2677   37   15   50  187   13   11
  258    3  159   24    5 2678   33   16  185   16    5]
[   1 2676  930    9  109  128 2677   37   15   50  187   13   11  258
    3  159   24    5 2678   33   16  185   16    5   63]
[2676  930    9  109  128 2677   37   15   50  187   13   11  258    3
  159   24    5 2678   33   16  185   16    5   63    6]
[ 930    9  109  128 2677   37   15   50  187   13   11  258    3  159
   24    5 2678   33   16  185   16    5   63    6 2679]
[   9  109  128 2677   37   15   50  187   13   11  258    3  159   24
    5 2678   33   16  185   16    5   63    6 2679   11]
[ 109  128 2677   37   15   50  187   13   11  258    3  159   24    5
 2678   33   16  185   16    5   63    6 2679   11  952]
[ 128 2677   37   15   50  187   13   11  258    3  159   24    5 2678
   33   16  185   16    5   63    6 2679   11  952  517]
[2677   37   15   50  187   13   11  258    3  159   24    5 2678   33
   16  185   16    5   63    6 26

In [29]:
# Get the last number (number 26) from each array. These will be our "y".
for sequence in num_sequences[:,-1]:
    print(sequence)

24
957
5
60
5
56
315
37
2
50
3
94
1
958
164
4
1
165
8
20
2
103
5
47
4
959
95
1
960
3
961
1
962
264
5
409
96
963
964
37
1
265
264
8
20
2
410
965
966
7
23
411
264
5
409
96
548
549
112
316
550
3
967
31
1
968
4
197
969
5
970
3
317
264
23
971
113
68
48
972
130
4
14
9
8
551
2
412
973
974
6
975
14
28
976
977
39
1
318
3
978
979
552
26
980
95
46
5
413
8
266
73
6
113
6
72
16
166
16
5
121
18
20
23
981
22
982
3
983
15
2
984
985
986
987
122
49
11
414
5
415
151
6
1
198
29
20
150
988
7
18
54
55
13
267
8
136
21
268
7
104
416
51
73
43
74
989
75
553
1
131
990
152
1
554
15
14
29
36
20
88
991
319
4
1
992
993
114
40
994
16
555
995
40
996
997
998
999
8
15
220
1000
221
3
417
1
178
151
19
556
222
1001
1002
20
1
557
115
9
558
1003
20
559
40
1004
3
1005
40
1006
69
2
320
269
321
41
42
4
322
4
179
223
17
1
560
4
167
1007
29
1008
1
319
4
2
1009
561
562
76
28
1010
1011
6
1012
1013
3
28
563
40
1014
1015
32
57
19
1016
53
564
1017
21
418
1
224
199
565
49
565
4
1018
268
419
7
554
566
51
567
168
1
1019
51
568
49
1
1020


In [30]:
# Set the input variable, `X`, to the first 25 numbers of each array.
X = num_sequences[:,:-1]
# Set target variable, `y`, to the last number of each array.
y = num_sequences[:,-1]

In [31]:
# Get the shape of X
print(X.shape)
# Get the length of each sequence.
seq_len = X.shape[1]
print(seq_len)

(11312, 25)
25


In [33]:
# Get the shape of y
y.shape

(11312,)

In [34]:
# Next,one-hot encode the target variable to get transform each index to a binary value. 
# We increase the vocabulary by 1 so we can predict the next word. 
y = to_categorical(y, num_classes=vocabulary_size+1)

In [35]:
# Get the shape of y again.
y.shape

(11312, 2719)

In [73]:
# Print the first 24 binary values in the first array.
# The index in for the last word in before transforming was "24", this is converted to a "1".
print(y[0,:25])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1.]


## Creating a LSTM  Model

In [37]:
# Import the dependencies for LSTM model.
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [38]:
def create_model(vocabulary_size, seq_len):
    """
    Create and compile an LSTM-based sequential model for text generation.

    Parameters:
    - vocabulary_size (int): The size of the vocabulary, i.e., the number of unique words in the text.
    - seq_len (int): The length of the input sequences, indicating the number of words in each sequence.

    Returns:
    - model (Sequential): A compiled Keras sequential model for text generation.

    Model Architecture:
    - Embedding Layer: Maps word indices to dense vectors.
    - LSTM Layer 1: 150 units, returns full sequence.
    - LSTM Layer 2: 150 units.
    - Dense Layer: 150 units, ReLU activation.
    - Output Layer: Dense layer with softmax activation for multi-class classification.

    Compilation:
    - Loss: Categorical crossentropy.
    - Optimizer: Adam.
    - Metrics: Accuracy.

    Usage Example:
    ```python
    model = create_model(vocabulary_size=10000, seq_len=25)
    ```
    """
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

## Training the Model

In [39]:
# Define the model and pass in the vocabulary (+1) and the seq_len (25 words).
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787394 (3.00 MB)
Trainable params: 787394 (3.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
# Fit model with 300 epochs. (This will take ~60 min to complete.)
model.fit(X, y, batch_size=128, epochs=300,verbose=1) 

Epoch 1/300
89/89 [==============================] - 17s 150ms/step - loss: 6.8459 - accuracy: 0.0469
Epoch 2/300
89/89 [==============================] - 13s 151ms/step - loss: 6.3859 - accuracy: 0.0529
Epoch 3/300
89/89 [==============================] - 13s 150ms/step - loss: 6.3378 - accuracy: 0.0529
Epoch 4/300
89/89 [==============================] - 13s 152ms/step - loss: 6.2009 - accuracy: 0.0530
Epoch 5/300
89/89 [==============================] - 13s 150ms/step - loss: 6.0940 - accuracy: 0.0537
Epoch 6/300
89/89 [==============================] - 13s 150ms/step - loss: 5.9577 - accuracy: 0.0614
Epoch 7/300
89/89 [==============================] - 13s 150ms/step - loss: 5.8541 - accuracy: 0.0631
Epoch 8/300
89/89 [==============================] - 13s 151ms/step - loss: 5.7577 - accuracy: 0.0665
Epoch 9/300
89/89 [==============================] - 13s 150ms/step - loss: 5.6727 - accuracy: 0.0727
Epoch 10/300
89/89 [==============================] - 13s 150ms/step - loss: 5.606

89/89 [==============================] - 14s 159ms/step - loss: 1.9564 - accuracy: 0.4906
Epoch 160/300
89/89 [==============================] - 14s 154ms/step - loss: 1.9491 - accuracy: 0.4914
Epoch 161/300
89/89 [==============================] - 14s 159ms/step - loss: 1.9354 - accuracy: 0.4910
Epoch 162/300
89/89 [==============================] - 14s 162ms/step - loss: 1.9267 - accuracy: 0.4987
Epoch 163/300
89/89 [==============================] - 14s 160ms/step - loss: 1.9086 - accuracy: 0.5027
Epoch 164/300
89/89 [==============================] - 14s 160ms/step - loss: 1.9011 - accuracy: 0.5073
Epoch 165/300
89/89 [==============================] - 14s 160ms/step - loss: 1.8962 - accuracy: 0.5041
Epoch 166/300
89/89 [==============================] - 15s 164ms/step - loss: 1.8733 - accuracy: 0.5114
Epoch 167/300
89/89 [==============================] - 15s 164ms/step - loss: 1.8672 - accuracy: 0.5122
Epoch 168/300
89/89 [==============================] - 15s 171ms/step - loss: 

In [41]:
from pickle import dump

In [42]:
# Save the model to file
# model.save('four_chapters_moby_dick_model_300.keras')
# Save the tokenizer
# dump(tokenizer, open('four_chapters_moby_dick_tokenizer_300', 'wb'))

## Generating New Text

In [43]:
# Import the dependencies needed for the LSTM.
from random import randint
from pickle import load
from keras.models import load_model
# pip install Keras-Preprocessing
from keras_preprocessing.sequence import pad_sequences

In [44]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    Generate text using a trained language model.
    
    INPUTS:
     - model: Trained language model (e.g., LSTM) capable of text generation.
     - tokenizer: Tokenizer that was fit on text data.
     - seq_len: Length of the training sequences used to train the model.
     - seed_text: A raw string text serving as the seed for text generation.
     - num_gen_words: The number of words to be generated by model.
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        # Pad sequences to our trained rate of 25 words.
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        # Predict Class Probabilities for each word
        pred_w = model.predict(pad_encoded, verbose=0)[0]
        
        pred_word_ind = np.argmax(pred_w, axis= -1)
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

## Test: Grab a random seed sequence

In [74]:
# Import the random module. 
import random

In [83]:
# Pick a random sequence of 26 words. 
random_pick = random.randint(10,len(text_sequences))
random_seed_text = text_sequences[random_pick]
# Join the words
seed_text = ' '.join(random_seed_text)
print(seed_text)

orion glitters what northern lights let them talk of their oriental summer climes of everlasting conservatories give me the privilege of making my own summer with


In [84]:
# Import the load_model method.
from keras.models import load_model

In [91]:
# Set the model to the saved trained 300 epoch model. 
model = load_model('four_chapters_moby_dick_model_300.keras')
# Set the tokenizer to the trained tokenizer from the model. 
tokenizer = load(open('four_chapters_moby_dick_tokenizer_300', 'rb'))

In [92]:
# Call the generate_text function and pass in the required parameters. We set the num_gen_words = 25. 
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'observing more and more the indecorous figure that queequeg made staving about with little else but his hat and boots on he begged him as'

- **The next 25 words aren't that accurate.**

## Explore Generating Text

In [88]:
# Choose a 26 word text string from the first four chapters of Moby Dick.
# If less than 26 the accuracy is off. 
text = """Seeing, now, that there were no curtains to the window, and that the street being very narrow,
the house opposite commanded a plain view into the room, and"""

In [89]:
# Create tokens by using the separate_punc function.
text_tokens = separate_punc(text)
print(text_tokens)
# Join the tokens and set them to the "seed_text" variable. 
seed_text = ' '.join(text_tokens)
print(seed_text)

['seeing', 'now', 'that', 'there', 'were', 'no', 'curtains', 'to', 'the', 'window', 'and', 'that', 'the', 'street', 'being', 'very', 'narrow', 'the', 'house', 'opposite', 'commanded', 'a', 'plain', 'view', 'into', 'the', 'room', 'and']
seeing now that there were no curtains to the window and that the street being very narrow the house opposite commanded a plain view into the room and


In [90]:
# Call the generate_text function and pass in the required parameters. We set the num_gen_words =50. 
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'observing more and more the indecorous figure that queequeg made staving about with little else but his hat and boots on he begged him as'

## Check the fist four chapters of Moby Dick to determine the accuracy of the text.
---
"Seeing, now, that there were no curtains to the window, and that the
street being very narrow, the house opposite commanded a plain view
into the room, and <font color='blue'>observing more and more the indecorous figure that
Queequeg made, staving about with little else but his hat and boots
on; I begged him as well as I could, to accelerate his toilet
somewhat, and particularly to get into his pantaloons as soon as
possible.</font>"

**Question: How would we gain better accuracy for the next 50 words?**

- Increase or decrease the length of the sequence? 
- Decrease the batch size? 